# Tutorial 6.2. Structural Effects of Wind Loading According to EN 1991-1-4

### Description: The wind effects on buildings (or generically any structure) according to EN 1991-1-4 is presented in this exercise. Along- and acrosswind responses according to EN 1991-1-4 and German national annex are addressed. Some exercises are proposed. 

#### Students are advised to complete the exercises. 

Project: Structural Wind Engineering WS19-20 
        Chair of Structural Analysis @ TUM - R. Wüchner, M. Péntek
        
Author: anoop.kodakkal@tum.de, mate.pentek@tum.de

Created on:  30.11.2015

Last update: 08.01.2020

##### Contents:

 1. Alongwind response of the structure
 2. Vortex shedding
 3. Instability - Galloping 

### Codes and standards:
#### 1. EN 1991-1-4:2005 - Eurocode 1: Actions on structures - Part 1-4: General actions - Wind actions
#### 2.  DIN EN 1991-1-4/NA:2010-12: Nationaler Anhang  EN 1991-1-4

Both can be downloaded from the links available on Moodle. It is advised to have them with you throughout this exercise. 

In [1]:
# import
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interactive

## 1. Alongwind response - computation of $c_s c_d$


#### Gust wind speed computed from the previous example for the location of the Jeddah Airport considering a return period of 50 years is 40.12 m/s . The mean wind speed is computed as $$ u_{mean} = u_{gust}/1.4$$  
The building is located at an urban area with height of adjacent building upto 15m: __Terrain category IV__

Let us take the same building as in previous example  

* height : 600 m 
* uniform cross section of given geometry and building width = 60.0 m
* assume a structural damping of 0,5%
* reference height set to be 200 m

In [2]:
mean_windspeed = 40.12 / 1.4
height_ref = 200
height = 600
width = 60
damping = 0.005
frequency_1 = 46 / height # refer to F.2

According to DIN EN 1991-1-4/NA the wind profile for terrain category IV is (ref: Tabelle NA.B.2)

$$ u_{mean}(z) = 0,56 . v_b . (z/10)^{0.3}$$ 

$$ u_{gust}(z) = 1,05 . v_b . (z/10)^{0.2}$$ 

Turbulence intensity $$ I_{v}(z) = 0,43 . (z/10)^{-0.3}$$ 

In [3]:
terrain_category = "IV"

a_mean  = {'I':1.18,'II':1.0,'III':0.77,'IV':0.56 }
alpha_mean = {'I':0.12,'II':0.16,'III':0.22,'IV':0.3 }
a_gust = {'I':1.61,'II':1.45,'III':1.27,'IV':1.05 }
alpha_gust = {'I':0.095,'II':0.12,'III':0.155,'IV':0.2 }
a_turb = {'I':0.14,'II':0.19,'III':0.28,'IV':0.43 }
alpha_turb = {'I':-0.12,'II':-0.16,'III':-0.22,'IV':-0.3 }

In [4]:
umean = a_mean[terrain_category] * mean_windspeed * (height_ref/10)**alpha_mean[terrain_category]
ugust = a_gust[terrain_category] * mean_windspeed * (height_ref/10)**alpha_gust[terrain_category]
Iv = a_turb[terrain_category]  * (height_ref/10)**alpha_turb[terrain_category]
print('Turbulence intensity', np.around(Iv,3))

Turbulence intensity 0.175


###### Computation of length scale L(z) (ref:  NA.C.1.2 & ref: Tabelle NA.C.1) . 

Length scale for terrain category IV  $$ L(z) = 300 .(z/300)^{0,46}$$ 

In [5]:
alpha_lengthscale = {'I':0.13,'II':0.26,'III':0.37,'IV':0.46}
len_scale = 300 * (height_ref/300) ** alpha_lengthscale[terrain_category]
print(len_scale)

248.95409571882422


###### Computation of the dimensionless power spectral density function SL(z,n) (ref:  NA.C.1.2). 

Dimensionless power spectral density function S_L(z,n) $$ S_L(z, n) = \dfrac{6,8 . f_L (z,n)}{(1+10,2 . f_L(z,n))^{5/3}}$$

where $f_L(z,n) = \dfrac{n . L(z)}{u_{mean}(z)}$ is the non-dimensional frequency

In [6]:
f_l = frequency_1 * len_scale / umean
S_l = 6.8 * f_l / (1+10.2 * f_l)** (5/3)
print(S_l)

0.16906048998718498


#### Determination of the background factor B² (ref: B.2) . 

the background factor can be calculated as  $$ B^2 = \dfrac{1}{1+0,9 . \bigg( \frac {b+h}{L(z)}\bigg)^{0,63}}$$


In [7]:
bg_factor = 1/(1+0.9*((width + height)/len_scale)**0.63)
print(bg_factor)

0.37545813068626666


###### Determination of the Resonance factor R² (ref: B.2)

the resonance factor can be calculated as  $$ R^2 = \dfrac{\pi^2}{2.\delta} . S_L(z,n). R_h(\eta_h). R_b(\eta_b) $$

refer to equations B5, B.7 and B.8 


In [8]:
eta_h = 4.6 * height / len_scale * f_l
eta_b = 4.6 * width / len_scale * f_l 
R_h = 1/eta_h - 1/2/eta_h**2 * ( 1 - np.e ** (-2 * eta_h))
R_b = 1/eta_b - 1/2/eta_b**2 * ( 1 - np.e ** (-2 * eta_b))
log_decrement = damping * 2 * np.pi
res_factor = np.pi**2 /2/log_decrement * S_l * R_h * R_b
up_cross_freq = max(frequency_1 * np.sqrt( res_factor / (res_factor + bg_factor)),0.08)
print(res_factor)

3.2337528286355393


###### Determination of the peak factor kp  (ref: B.2)

refer to equation B.4

In [9]:
peak_factor = max(3, np.sqrt(2 * np.log(up_cross_freq * 600 )) + 0.6 / (np.sqrt(2*np.log(up_cross_freq * 600))))
print(peak_factor)

3


###### Determination of structural factor $C_sC_d$

refer to equation NA.C.1 
 $$ c_s c_d = \dfrac{1+2 . k_p . I_v(z) . \sqrt{B^2 + R^2 } } {1+6. I_v(z)} $$

In [10]:
structural_factor = (1 + 2*peak_factor*Iv*np.sqrt(bg_factor+res_factor))/(1+6 * Iv)
print('Structural factor c_s c_d is ', np.around(structural_factor, 3))

Structural factor c_s c_d is  1.461


### Exercise: Where is the $c_sc_d$ is used ? 

Refer to equation 5.4 in DIN EN 1991-1-4

For the exercise 3_1_wind_load, recompute the bending moment and shear force considering this factor.
Knowing the structural properties compute the static displacements for this case. 

## 2. Vortex shedding 

Refer to Annex E 

The effect of vortex shedding needs to be investigated when $V_{crit} < 1,25 . V_m$


The critical wind velocity is $V_{crit} = \dfrac{b . n_{i,y}}{St}$

Refer to E1.3.1 for details 

In [11]:
strouhal_number = 0.2 # for rectangular cross section # refer to E1.3.2 for details 

crosswind_freq = 0.26 # this refers to the frequency correspondign to th bending mode in crosswind direction
velocity_crit = width * crosswind_freq / strouhal_number

In [12]:
vortex_freq = strouhal_number * umean / width
print('Vortex shedding Frequency ', np.around(vortex_freq,3))

Vortex shedding Frequency  0.131


In [13]:
if (1.25 * umean > velocity_crit): 
    print('Not OK, The effect of vortex shedding need to be investigated')
else: 
    print('OK, The effect of vortex shedding does not have to be investigated')

OK, The effect of vortex shedding does not have to be investigated


##### Calculation of acrosswind amplitude of displacements - Scruton number Sc

The susceptibility of vibrations depends on the structural damping and the ratio of structural mass
to fluid mass. This is expressed by the Scruton number Sc, which is given in Expression E.4 

 $Sc = \dfrac{2.\delta_{s}.m_{i,e}}{\rho . b^2}$



In [14]:
# Assumed structural properties 
concrete_density = 2500 # kg/m**3
area = 12.50 # m**2
breadth = 4.75 #m
depth = 9.5 # m
frequency_1 = 0.24 # Hz
height = 70.0 # m 
rho_air= 1.25
damping_ratio = 0.005 

As an initial value one could use the total mass (the way it is used in the next block), for a more correct evaluation one should use the modal mass. For a cantilever-type structure refer to pre-calculated values for effective modal mass, such as equation D-70 in [Vibration Data (by Tom Irvine)](http://www.vibrationdata.com/tutorials2/beam.pdf).

In [15]:
m_e = concrete_density * area
log_decrement = damping_ratio * 2 *np.pi
scruton_number = 2 * log_decrement * m_e / rho_air / breadth **2
print(scruton_number)

69.61978179700372


##### Computation of lateral force coefficient $C_{lat,0}$

From Table E.2 

In [16]:
c_lat_0 = 1.1

From Table E.3

In [17]:
c_lat = c_lat_0

correlation length factor $K_W$ and shape factor $K$ From Table E.5

In [18]:
K = 0.13
#From table E.4
#L_j/b = 6
K_w = (3*6/(height/breadth))*(1-(6/(height/breadth))+1/3*(6/(height/breadth))**2)
print(K_w)

0.7916228134110788


##### Calculation of acrosswind amplitude of displacements

ref E.1.5.2.1 : the largest displacement $y_{F,max}$ can be calculated by the expression (E.7) as 

$\dfrac{y_{F,max}}{b} = \dfrac{1}{St^2}.\dfrac{1}{Sc}.K.Kw.c_{lat}$

where, $Sc$ is the Scruton number as in E.1.3.3. 

Refer to section E.1.5.2 for more details

In [19]:
y_f_max = breadth/scruton_number /strouhal_number**2 * K * K_w * c_lat
print('maximum y-displacement in across wind direction in m is ',np.around(y_f_max,3))

maximum y-displacement in across wind direction in m is  0.193


Assuming a SDoF-similarity (puerly harmonic oscillatory displacement lead to purely harmonic velocities and accelerations due to derivation with respect to time), maximum acceleration can be obtained as $-\omega^2 * y_{F,max}$

In [20]:
acc_f_max = frequency_1**2/4*np.pi**2  * y_f_max
print('maximum y-acceleration in acrosswind direction in m/s² is ', np.around(acc_f_max,3))

maximum y-acceleration in acrosswind direction in m/s² is  0.027


## 3. Galloping

Refer to Annex E.2

The onset wind velocity of galloping is given by $$ V_{cg} = \dfrac{2.Sc}{a_G} . n_{1y}.b$$

In [21]:
#from table E.7
a_g = 2.0
v_cg = 2 * scruton_number * frequency_1 * breadth / a_g 
print(v_cg)

79.36655124858424


#### Discuss among groups the likelihood of different wind effects on the structure. 